In [7]:
from binance.client import Client
import numpy as np
import ta
from backtesting import Strategy, Backtest
import pandas as pd
client = Client('ILBDb6uATI7e50vDp7NtPiKhnonJ1ubDxJW7DScLkih2eaBoW0Wc3sIVpArbx40K', 'ICeBRRrLX5rHNIIOHPyOX2coxqEVk1yPgTN44briSVjhrk1xutTtDTqtyERLzMCu') 

In [8]:
klines30 = np.array(client.get_historical_klines('BTCUSDT', '4h',  "1 Jan, 2015", "31 Dec, 2020"))

def binanceDataFrame( klines):
    df = pd.DataFrame(klines.reshape(-1,12),dtype=float, columns = ('Open Time',
                                                                    'Open',
                                                                    'High',
                                                                    'Low',
                                                                    'Close',
                                                                    'Volume',
                                                                    'Close time',
                                                                    'Quote asset volume',
                                                                    'Number of trades',
                                                                    'Taker buy base asset volume',
                                                                    'Taker buy quote asset volume',
                                                                    'Ignore'))

    df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
    return df

data = binanceDataFrame(klines30)[['Open Time', 'Open', 'High', 'Low', 'Close']]
data['RSI'] = ta.momentum.RSIIndicator(data['Close'], 2, False).rsi()
data['SMA5'] = ta.trend.SMAIndicator(data['Close'], 5, False).sma_indicator()
data['SMA200'] = ta.trend.SMAIndicator(data['Close'], 80, False).sma_indicator()
data['MIN10'] = data['Close'].rolling(10).min().shift(1)
data['MAX20'] = data['Close'].rolling(20).max().shift(1)
data.set_index('Open Time', inplace=True)
data.dropna(inplace=True)

In [9]:
data

,Open,High,Low,Close,RSI,SMA5,SMA200,MIN10,MAX20
Open Time,,,,,,,,,
2017-08-30 08:00:00,4550.00,4597.00,4416.01,4498.35,13.551236,4555.002,4223.240250,4321.65,4587.48
2017-08-30 12:00:00,4537.00,4592.76,4416.16,4546.77,60.397729,4553.246,4225.700000,4329.02,4587.48
2017-08-30 16:00:00,4555.00,4609.92,4532.80,4536.82,49.396467,4543.114,4227.069000,4329.02,4587.48
2017-08-30 20:00:00,4536.82,4579.91,4516.87,4555.14,69.711906,4537.420,4229.604000,4329.02,4587.48
2017-08-31 00:00:00,4555.14,4606.00,4555.14,4593.66,88.733146,4546.148,4232.959375,4368.50,4587.48
...,...,...,...,...,...,...,...,...,...
2020-12-30 08:00:00,28389.02,28579.92,27320.00,27838.82,42.586541,27721.520,24480.688375,26435.63,28386.62
2020-12-30 12:00:00,27836.95,28291.82,27577.00,28153.76,65.148527,27965.948,24551.625875,26435.63,28386.62
2020-12-30 16:00:00,28153.75,28900.05,27887.00,28776.46,86.354039,28244.240,24626.929625,26435.63,28386.62


In [10]:
class IFRBacktest(Strategy):
    def init(self):
        pass

    def next(self):
        price = self.data.Close[-2]
        rsi = self.data.RSI[-2]
        sma5 = self.data.SMA5[-2]
        sma200 = self.data.SMA200[-2]
        if not self.position:
            if rsi < 10 and (price > sma200):
                self.buy()
            '''elif rsi > 90:
                self.sell()'''
        else:
            if self.position.is_long and (price > sma5):
                self.position.close()
            elif self.position.is_short and (price < sma5):
                self.position.close()

In [11]:
class Turtle(Strategy):
    def init(self):
        pass

    def next(self):
        price = self.data.Close[-2]
        rsi = self.data.RSI[-2]
        min10 = self.data.MIN10[-2]
        max20 = self.data.MAX20[-2]
        if not self.position:
            if price < min10:
                self.buy()
        else:
            if self.position.is_long and (price > max20):
                self.position.close()

In [12]:
bt = Backtest(data, IFRBacktest, commission=0, exclusive_orders=False)
print(bt.run())
bt.plot()

/tmp/ipykernel_19366/4214943948.py:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, IFRBacktest, commission=0, exclusive_orders=False)


Start                     2017-08-30 08:00:00
End                       2020-12-31 00:00:00
Duration                   1218 days 16:00:00
Exposure Time [%]                    7.126216
Equity Final [$]                     12575.36
Equity Peak [$]                      12780.36
Return [%]                            25.7536
Buy & Hold Return [%]              545.214356
Return (Ann.) [%]                    7.096321
Volatility (Ann.) [%]               11.407993
Sharpe Ratio                         0.622048
Sortino Ratio                         0.94409
Calmar Ratio                          0.68891
Max. Drawdown [%]                   -10.30079
Avg. Drawdown [%]                   -2.575251
Max. Drawdown Duration      560 days 16:00:00
Avg. Drawdown Duration       59 days 07:00:00
# Trades                                  102
Win Rate [%]                        61.764706
Best Trade [%]                       4.907893
Worst Trade [%]                    -10.353852
Avg. Trade [%]                    

Row(id='1351', ...)

In [13]:
bt = Backtest(data, Turtle, commission=0, exclusive_orders=False)
print(bt.run())
bt.plot()

/tmp/ipykernel_19366/3644405135.py:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Turtle, commission=0, exclusive_orders=False)


Start                     2017-08-30 08:00:00
End                       2020-12-31 00:00:00
Duration                   1218 days 16:00:00
Exposure Time [%]                     47.9101
Equity Final [$]                      7951.38
Equity Peak [$]                      17507.75
Return [%]                           -20.4862
Buy & Hold Return [%]              545.214356
Return (Ann.) [%]                   -6.628495
Volatility (Ann.) [%]               48.809119
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -67.152718
Avg. Drawdown [%]                   -7.210412
Max. Drawdown Duration     1088 days 04:00:00
Avg. Drawdown Duration       42 days 15:00:00
# Trades                                  114
Win Rate [%]                        65.789474
Best Trade [%]                      18.825402
Worst Trade [%]                    -36.724909
Avg. Trade [%]                    

Row(id='1942', ...)

In [14]:
data

,Open,High,Low,Close,RSI,SMA5,SMA200,MIN10,MAX20
2017-08-30 08:00:00,4550.00,4597.00,4416.01,4498.35,13.551236,4555.002,4223.240250,4321.65,4587.48
2017-08-30 12:00:00,4537.00,4592.76,4416.16,4546.77,60.397729,4553.246,4225.700000,4329.02,4587.48
2017-08-30 16:00:00,4555.00,4609.92,4532.80,4536.82,49.396467,4543.114,4227.069000,4329.02,4587.48
2017-08-30 20:00:00,4536.82,4579.91,4516.87,4555.14,69.711906,4537.420,4229.604000,4329.02,4587.48
2017-08-31 00:00:00,4555.14,4606.00,4555.14,4593.66,88.733146,4546.148,4232.959375,4368.50,4587.48
...,...,...,...,...,...,...,...,...,...
2020-12-30 08:00:00,28389.02,28579.92,27320.00,27838.82,42.586541,27721.520,24480.688375,26435.63,28386.62
2020-12-30 12:00:00,27836.95,28291.82,27577.00,28153.76,65.148527,27965.948,24551.625875,26435.63,28386.62
2020-12-30 16:00:00,28153.75,28900.05,27887.00,28776.46,86.354039,28244.240,24626.929625,26435.63,28386.62
2020-12-30 20:00:00,28776.45,28996.00,28534.70,28875.54,88.567646,28406.240,24696.201375,26435.63,28776.46


In [15]:
!pip install binance-connector


Defaulting to user installation because normal site-packages is not writeable


In [16]:
pip install binance-connector


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install backtesting

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
